<a href="https://colab.research.google.com/github/Justin-Jonany/FLARE_Implementation/blob/main/notebooks_and_demonstration/1_FLARE_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FLARE**
We know that when LLM's output tokens, they also output the probabilities associated with each token, as a result of the transformer model's outputting layer. However, what does these number mean? There have been several researchs arround this, some takes these numbers as how confident the LLMs are with its answer. This means that what if for every token that an LLM outputs with low probability, we assist the LLM to make it sure.

Introducing, the paper "[Active Retrieval Augmented Generation](https://arxiv.org/pdf/2305.06983)" by Jiang ZB and fellow researchers written on October 22, 2023, and the goal of this project will be to implement the paper and test it for various tasks. Although, at the the time of this project's starting date (August 6, 2024), it may seem relatively out-of-date, this technique is still relevant and efficient, and can be implemented for any future model that allows users to access the probabilities of each tokens.

All functions can be accessed in my github repository: [github.com/Justin-Jonany/FLARE_Implementation](https://github.com/Justin-Jonany/FLARE_Implementation)

## **Summary of the Paper (outputted by my implementation of FLARE)**
The paper explores the limitations of existing retrieval-augmented language models, which typically execute a single retrieval based solely on user input, leading to inaccurate and contextually irrelevant outputs. Recognizing the need for continual information gathering to enhance long-form text generation, the authors propose Forward-Looking Active Retrieval Augmented Generation (FLARE).

This method employs an iterative approach, where the model generates a provisional next sentence to identify low-confidence tokens and selectively retrieves relevant information to refine its output. FLARE operates through two main strategies: FLAREinstruct, which prompts the model to create retrieval queries as needed, and FLAREdirect, where the model’s generative output directly informs search queries. The authors empirically validate FLARE across multiple knowledge-intensive tasks and demonstrate its superior performance compared to traditional retrieval methods, emphasizing its effectiveness in enriching the generation process with accurate, contextually rich information that adapts to the evolving needs of text generation.

## **My Implementation**
I'm going to implement the FLARE Instruct version of the paper based on the paper's algorithm structure and will not be based-on the paper's source code. I will additionally use GPT-4o instead of GPT 3.5. The retriever can be any object that has a method called "get_relevant_documents". For this notebook, the retriever is Chroma with a langchain OpenAI Embedding.

The steps of the RAG are the following:
### 1. Use the LLM for traditional RAG or regular querying to answer the question
As the first output of the recursive steps, the question will be invoked on the LLM given a context retrieved from a basic RAG call to the retriever.

### 2. Check logits for each token from the LLM and annotate with a symbol any where the llm is not confident.
The next step is to check each token's logarithmic probabilities, and for a given tolerance, a function will automatically  modify the OpenAI Response object's logprob field with annotations. For every phrase where the it's unsure, it will be marked as the following: "[uncertain]...[/uncertain]". This is done with the assumption, that if the LLM outputs token with low probabilities, it implies that it's unsure about it, meaning it has a higher chance to be wrong.

### 3. Constructing questions to get a more confident answer for that token.
Now for all phrases that's marked as uncertain, the LLM will be reinvoked to turn the "[uncertain]...[/uncertain]" into "[search(question)]", where "question" would be the prompt to answer the uncertain phrase. After, these questions will be extracted into a dictionary, to ease the question-answering process.

### 4. Answer these questions
Now, having the questions, the LLM will be used to answer each question with a prompt of the following format: a shortened version of the steps so far, the annotated answer, the question, and finally the context retrieved. All answers will be saved a in dictionary as the value, and the question as the key.

### 5. Reconstruct the answer
Now having, all the answers, the LLM will be used the reconstruct the final answer.

## **Example Steps**
For the question: Why did Arkad believe that good luck follows opportunity?. My implementaton of FLARE would look like this:

### Step 1. Regular RAG Answer
In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them.

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success.

### Step 2. Annotating uncertain tokens
In "The Richest Man in ([uncertain] Babylon," Arkad, who is the [/uncertain])  richest man in Babylon, believes that good luck follows opportunity because he sees luck as ([uncertain] a byproduct of one’s readiness [/uncertain])  to seize opportunities when they arise. ([uncertain] He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the [/uncertain])  opportunities presented to them.

 ([uncertain]Arkad suggests that those who are diligent, prepared, and willing to work toward their goals [/uncertain])  are more likely to encounter ([uncertain] opportunities that lead to success. [/uncertain])  ([uncertain] In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take [/uncertain])  risks, and the ability to ([uncertain] recognize and act on chances [/uncertain])  ([uncertain] that life presents. [/uncertain])  ([uncertain] This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial [/uncertain])  success.

 ### Step 3. Constructing questions to fix the low-confidence tokens
 In "The Richest Man in [Search(What is the setting of the book "The Richest Man in Babylon"?)], Arkad, who is the [Search(Who is Arkad in the context of the book?)] richest man in Babylon, believes that good luck follows opportunity because he sees luck as [Search(What does Arkad mean by "a byproduct of one’s readiness" in terms of seizing opportunities?)] to seize opportunities when they arise. [Search(Why do people miss chances to become wealthy according to Arkad?)] He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the [Search(What type of opportunities does Arkad refer to in the book?)] opportunities presented to them.

[Search(What characteristics do diligent, prepared, and hardworking people exhibit according to Arkad?)] Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter [Search(What are examples of opportunities that lead to success in the book?)] opportunities that lead to success. [Search(What does Arkad say about the nature of luck?)] In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take [Search(What type of risks does Arkad encourage people to take?)] risks, and the ability to [Search(How does one recognize and act on chances?)][Search(What does Arkad mean by "chances" in the context of life?)] recognize and act on chances [Search(What does Arkad mean by "that life presents"?)] that life presents. [Search(Why does Arkad advocate for a proactive approach to achieving financial success?)] This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial [Search(What does financial success mean in the context of the book?)] success.

### Step 4. Extract as dictionary
{'1': "What is the setting of the book 'The Richest Man in Babylon'?",
 '2': 'Who is Arkad in the context of the book?',
 '3': "What does Arkad mean by 'a byproduct of one’s readiness' in terms of seizing opportunities?",
 '4': 'Why do people miss chances to become wealthy according to Arkad?',
 '5': 'What type of opportunities does Arkad refer to in the book?',
 '6': 'What characteristics do diligent, prepared, and hardworking people exhibit according to Arkad?',
 '7': 'What are examples of opportunities that lead to success in the book?',
 '8': 'What does Arkad say about the nature of luck?',
 '9': 'What type of risks does Arkad encourage people to take?',
 '10': 'How does one recognize and act on chances?',
 '11': "What does Arkad mean by 'chances' in the context of life?",
 '12': "What does Arkad mean by 'that life presents'?",
 '13': 'Why does Arkad advocate for a proactive approach to achieving financial success?',
 '14': 'What does financial success mean in the context of the book?'}

 ### Step 5. Answering each question
 1. **Question**: What is the setting of the book 'The Richest Man in Babylon'?\
**Answer**: Arkad believed that good luck follows opportunity because he understood that wealth is not simply a matter of fate or luck; it is a result of taking action when opportunities present themselves. During his journey to accumulate wealth, he realized that many people fail to recognize the opportunities available to them or choose not to act on them. Instead of waiting for fortune to smile upon them, Arkad emphasized the importance of being proactive, diligent, and prepared to seize chances when they arise.\
According to Arkad, those who actively seek opportunities, educate themselves about how to manage and grow their wealth, and are willing to work and invest wisely are more likely to create their own "luck." This philosophy is encapsulated in his advice that a part of all one earns should be kept for oneself, which reflects the notion that individuals have the power to influence their financial destiny by being open to opportunity and ready to take positive steps toward wealth accumulation.

2. Question: Who is Arkad in the context of the book?\
Answer: Arkad believed that good luck follows opportunity because he understood that those who are prepared and willing to act upon the opportunities that life presents are more likely to find success. Essentially, luck, in Arkad's view, is not a mere coincidence; instead, it is a result of one’s readiness to seize chances and make the most of them. He recognized that many people fail to achieve wealth because they do not identify or take action on the opportunities available to them.\
Moreover, Arkad’s own experiences shaped this philosophy. He started his own fortune in humble beginnings and learned valuable lessons that he later shared with others in Babylon. His teachings emphasized that by being proactive, recognizing potential opportunities, and being willing to take calculated risks, individuals could attract what they perceive as "good luck." Thus, in Arkad's perspective, the consistent effort towards preparation and action creates the conditions for good luck to manifest.

...other question-answer pairs...

### Step 6. Reconstructing the final answer:
In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he understands that wealth is not simply a matter of fate or luck; it is a result of taking action when opportunities present themselves. Throughout his life, he observed that many people fail to recognize the opportunities available to them or choose not to act on them. Instead of waiting for fortune to smile upon them, Arkad emphasized the importance of being proactive, diligent, and prepared to seize chances when they arise.

According to Arkad, those who actively seek opportunities, educate themselves about how to manage and grow their wealth, and are willing to work and invest wisely are more likely to create their own "luck." He noted that while opportunities are available to everyone, only a few grasp them and achieve their desires, while the majority hesitate or falter and consequently fall behind. This perspective underscores that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on possibilities that life presents.

In essence, Arkad's philosophy encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. He conveys that good luck is closely linked to the willingness to engage with life's opportunities, reinforcing the idea that preparation and action can transform luck from a mere chance event into a consistent part of one's journey towards success.



## **FLARE Use-Cases**
In the following notebooks, FLARE will be used for various tasks: Fund Statement PDF Extraction, Novel Question-Answering, and Scientific Journal Summarization.

# Libraries

In [ ]:
!git clone https://github.com/Justin-Jonany/FLARE_Implementation.git

Cloning into 'FLARE_Implementation'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 242 (delta 126), reused 177 (delta 72), pack-reused 0 (from 0)
Receiving objects: 100% (242/242), 6.92 MiB | 13.96 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [ ]:
pip install -r FLARE_Implementation/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
pip install chromadb openai langchain langchain_chroma langchain_community langchain_core langchain_openai langchain_text_splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 126.7 MB/s eta 0:00:00
 

In [ ]:
import os
from os import listdir
from os.path import isfile, join
from random import sample
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chromadb
import copy
from openai import OpenAI
from IPython.display import Markdown
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from math import floor, ceil
import ast

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

# Data

In [ ]:
df_metadata = pd.read_csv('FLARE_Implementation/rmib_dataset/rmib_metadata.csv')
df_metadata

,chars,words,lines,title,type,text
0,3887,721,63,the sixth cure,chapter,"THE SIXTH CURE \nInsure a future income \n\n""T..."
1,6696,1177,118,the walls of babylon,chapter,"The Walls of Babylon \n\nOld Banzar, grim warr..."
2,459790,83596,8311,metadata,chapter,",chars,words,lines,title,type,text\n9_the_sixt..."
3,3728,665,60,the third cure,chapter,"THE THIRD CURE \nMake thy gold multiply \n\n""B..."
4,1046,201,77,table of contents,table of contets,Table of Contents \n\nAbout the author 3 \n\nF...
5,830,134,14,about the author,about the author,About the author \n\nGEORGE SAMUEL CLASON was ...
6,1890,306,29,foreword,foreword,Foreword \n\nOur prosperity as a nation depend...
7,2675,471,42,the fourth cure,chapter,THE FOURTH CURE \nGuard thy treasures from los...
8,5006,916,96,the seventh cure,chapter,THE SEVENTH CURE \nIncrease thy ability to ear...
9,3901,694,74,the second cure,chapter,THE SECOND CURE \nControl thy expenditures \n\...


In [ ]:
def flare(question, retriever, openai_api_key, openai_model='gpt-4o-mini', tolerance=-0.4, verbose=True):
    '''
    Uses a advanced RAG technique called FLARE to answer the question. It's an implementation
    of the paper: "Active Retrieval Augmented Generation" Jiang ZB and fellow scientists in October
    2023.

    Args:
      question: The question to be answered.
      retriever: langchain retriever to be used.
      openai_api_key: The OPENAI API key to be used.
      openai_model: OpenAI Model to use
      tolerance: The tolerance of logprobs to be marked as uncertain
    Returns:
      The answer to the question.

    '''
    client = OpenAI(api_key=openai_api_key)

    # getting the first output, normal rag
    # get context
    context = retriever.get_relevant_documents(question)

    # constructing message
    message = [
        {"role": "system", "content": """You are a book expert that answers questions about books. Use the following pieces of retrieved context to answer the question."""},
        {"role": "user", "content": f"""
        Context: {format_docs(context)}
        Question: {question}
        Answer:
        """}
    ]

    if verbose: print(f'Acquiring answer with traditional RAG...')
    # answer the question
    answer = client.chat.completions.create(
        model=openai_model,
        messages=message,
        logprobs=True,
    )

    if verbose: print(f'Finding uncertain tokens, and annotating the answer...')
    # annotate the question
    annotated_answer = uncertain_marker(annotated_combiner(annotater(sequential_combine(
        combine_token_to_word(answer), 5, np.mean), tolerance=tolerance), np.mean))

    # constructing the questions for the uncertained answers
    message += [
        {"role": "assistant", "content": answer.choices[0].message.content},
        {"role": "system", "content": f"""Now, I have marked the answer to where you are uncertain with the phrases. For every, phrases in between
        [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)].

        This question is going to be used independently to get get relevant texts from a vector database
        It's critical that the question includes the object and subject of the phrase
        It's critical that the question has context about the annswer


        First example:
        user: What is meaning of the colors in the flag of Ghana?
        assistant: Red is for the blood of martyrs, green for forests, and gold for mineral wealth.
        user: Here's the annotated version: ([uncertain] Red [/uncertain]) is for the blood of martyrs, ([uncertain] green for forests [/uncertain]), and gold for mineral wealth.
        assistant: [Search(is red a color in the flag of Ghana?)] is for the blood of martyrs, [Search(is green a color in the flag of Ghana? If so, what does it symbolize?)], and gold for mineral wealth.

        Second example:
        user: Give me a very short summary of Joe Biden's journey becoming the president!
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on August 18, 2019. His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        user: Here's an annotated version: Joe Biden announced his candidacy for the 2020 presidential election on ([uncertain] August 18, 2019 [/uncertain]). His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on [Search(When did Joe Biden announce his candidancy for the 2020 presidential election?)].  His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        """},
        {"role": "user",
            "content": f"Here's the annotated version: {annotated_answer.choices[0].message.content}"},
    ]

    if verbose: print(f'Constructing questions for the annotated tokens...')
    questions_construction = client.chat.completions.create(
        model=openai_model,
        messages=message,
    )

    # extracting the questions
    message += [
        {"role": "assistant",
            "content": questions_construction.choices[0].message.content},
        {'role': "user", "content": """Now for all the questions marked as [Search(question)], please extract them in a python dictionary format:
        {
        "1": "question 1",
        "2": "question 2",
        ...
        "n": "question n"
        }

        It is critical to only output the dictionary and nothing else.
        It is critical to not output it in a markdown format.
        It is critical that the first character of the output starts with an open curly bracket '{'
      """}
    ]

    if verbose: print(f'Extracting constructed questions...')
    questions = client.chat.completions.create(
        model=openai_model,
        messages=message,
    )
    retry_count= 1
    try:
        questions_dict = ast.literal_eval(questions.choices[0].message.content)
    except:
        while retry_count <= 3:
            if verbose: print(f"Couldn't convert to dictionary, attempting to fix the dictionary. Retry count: {retry_count}")
            questions = client.chat.completions.create(
                model=openai_model,
                messages=message,
            )
            try:
                questions_dict = ast.literal_eval(questions.choices[0].message.content)
                break
            except:
                retry_count += 1
                continue
        else:
            print(f'FLARE Failed, try to call the function again.')
            return

    # message to answer the questions one by one
    new_message = [
        {"role": "system", "content": f"""You are a book expert that answers questions about books.
      Question: {question}
      Context: called the RAG
      Original Answer: {answer.choices[0].message.content}

      Now, I have marked the answer to where you are uncertain with the phrases. For every, phrases in between
      [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)]

      Annotated Answer: {annotated_answer.choices[0].message.content}
      Constructed Questions Answer: {questions_construction.choices[0].message.content}
      """},
    ]

    if verbose: print(f'Answering each questions...')
    # answering each of the questions one by one
    constructed_question_answer = {}
    for i in range(1, len(questions_dict) + 1):
        question_temp = questions_dict[str(i)]

        # getting the context for the question
        context = retriever.get_relevant_documents(question_temp)

        # constructing the question and context message
        question_string = f"""Use the following pieces of retrieved context to answer the question.
        Question: {question}
        Context: {format_docs(context)}
        Answer:
        """
        question_message = new_message + \
            [{"role": "user", "content": question_string}]

        # answering the question
        question_answer = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=question_message,
        )
        constructed_question_answer[str(i)] = [
            question_temp, question_answer.choices[0].message.content]

    # reconstructing to get the final answer
    question_answer = ""
    for i in range(1, len(questions_dict) + 1):
        question_answer += f"""Question: {questions_dict[str(i)]}\nAnswer: {constructed_question_answer[str(i)][1]}\n"""

    reconstructing = f"""
    Here are the questions and their answers:
    {question_answer}
    Now with answers to those questions and the original question: {question}, improve the original answer without changing the format of the answer.

    Notes:
    It's critical to just output the final answer.
    It's critical to not output the annotated answer.
    It's critical to not output constructed questions answer.

    Final Answer:
    """
    reconstructing_message = new_message + \
        [{"role": "user", "content": reconstructing}]
    if verbose: print(f'Reconstructing the final answer...')
    reconstructed_answer = client.chat.completions.create(
        model=openai_model,
        messages=reconstructing_message,
    )

    # printing some information
    # It prints the question construction for the annotated answer because it's clearer to make the annotations as the questions itself
    return answer, questions_construction, reconstructed_answer = pd.read_csv('FLARE_Implementation/questions', index_col=['chapter', 'index'])

In [ ]:
question = df_questions.loc['meet the goddess of good luck', 0]['hard']
print(question)

Why did Arkad believe that good luck follows opportunity?


# FLARE Implementation
From the paper, here's the approaches I'm going to make based on the paper.
1. Use the LLM for traditional RAG or regular querying to answer the question
2. Check logits for each token from the LLM and annotate with a symbol any where the llm is not confident.
3. Constructing questions to get a more confident answer for that token.
4. Answer these questions
5. Reconstruct the Answer

## Helper Functions

In [ ]:
class Fake_Retriever:
  '''
  Instead of a vector database. FLARE can be used for regular zero-shot
  prompting. If the context is relatively not too long, it may be better
  to just give the whole context to the LLM at all cases.

  The class Fake_Retriever returns the whole context everytime a retriever
  is called.
  '''
  def __init__(self, data):
    self.data = Document(page_content=data)
  def get_relevant_documents(self, query):
    return [self.data]


def logprobs_simple_print(response, n=None):
    """
    Prints an OpenAI response with logprobs for each tokens

    Args:
      response: OpenAI Response object with logprobs
      n: First n tokens to print
    """
    for i, token_data in enumerate(response.choices[0].logprobs.content[:n]):
        print(f'{i}: {token_data.token}')
        print(token_data.bytes)
        if hasattr(token_data, 'uncertain'):
            print(f'{token_data.logprob}\t{str(token_data.uncertain)}')
        else:
            print(f'{token_data.logprob}')
        print('\n')


def logprobs_pretty_print(response, prompt):
    """
    Pretty prints an OpenAI response with logprobs for each tokens

    Args:
      response: OpenAI Response object with logprobs
      prompt: Original Prompt
    """
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    response_text = response.choices[0].message.content
    response_text_tokens = [
        token.token for token in response.choices[0].logprobs.content]
    if hasattr(response.choices[0].logprobs.content[0], 'uncertain'):
        uncertainty = [
            token.uncertain for token in response.choices[0].logprobs.content]

    max_starter_length = max(
        len(s) for s in ["Prompt:", "Response:", "Tokens:", "Logprobs:", "Perplexity:"])
    max_token_length = max(len(s) for s in response_text_tokens)

    new_lines_index = []
    for i, token in enumerate(response_text_tokens):
        if '\n' in token:
            new_lines_index += [i]
    formatted_response_tokens = [
        s.rjust(max_token_length) for s in response_text_tokens]
    formatted_lps = [f"{lp:.2f}".rjust(max_token_length) for lp in logprobs]
    formatted_linear_probs = [
        f"{np.round(np.exp(lp)*100,2):.2f}%".rjust(max_token_length) for lp in logprobs]
    if hasattr(response.choices[0].logprobs.content[0], 'uncertain'):
        formatted_uncertain = [str(uncertain).rjust(
            max_token_length) for uncertain in uncertainty]
    perplexity_score = np.exp(-np.mean(logprobs))
    print("Prompt:".ljust(max_starter_length), prompt)
    print("Response:".ljust(max_starter_length), response_text, "\n")
    print("=" * 150)
    cut_off_start = 0
    cut_off_end = 0
    for i, new_line_index in enumerate(new_lines_index):
        cut_off_start = 0 if i == 0 else new_lines_index[i - 1] + 1
        cut_off_end = new_lines_index[i] + 1
        print("Tokens:".ljust(max_starter_length), " ".join(
            formatted_response_tokens[cut_off_start:cut_off_end]))
        print("Logprobs:".ljust(max_starter_length), " ".join(
            formatted_lps[cut_off_start:cut_off_end]))
        print("Linprob:".ljust(max_starter_length), " ".join(
            formatted_linear_probs[cut_off_start:cut_off_end]))
        if hasattr(response.choices[0].logprobs.content[0], 'uncertain'):
            print("Uncertainty:".ljust(max_starter_length), " ".join(
                formatted_uncertain[cut_off_start:cut_off_end]))
        print("=" * 150)
    print("Tokens:".ljust(max_starter_length), " ".join(
        formatted_response_tokens[cut_off_end:]))
    print("Logprobs:".ljust(max_starter_length),
          " ".join(formatted_lps[cut_off_end:]))
    print("Linprob:".ljust(max_starter_length),
          " ".join(formatted_linear_probs[cut_off_end:]))
    if hasattr(response.choices[0].logprobs.content[0], 'uncertain'):
        print("Uncertainty:".ljust(max_starter_length),
              " ".join(formatted_uncertain[cut_off_end:]))

    print("=" * 150)
    print("Perplexity:".ljust(max_starter_length), perplexity_score, "\n")


def format_docs(docs):
    '''
    formats the list of documents into a string

    Args:
      docs: list of documents

    Returns:
      string of documents
    '''
    return "\n\n".join(doc.page_content for doc in docs)

def combine_token_to_word(response):
    """
    Combines the tokens in an OpenAI response that are parts of words, into a word.

    Args:
        response: OpenAI response object

    Returns:
        Open AI Response object
    """
    temp_response = copy.deepcopy(response)
    new_logprobs_list = []
    new_logprob = temp_response.choices[0].logprobs.content[0]
    skip_next = False
    for i, token_data in enumerate(temp_response.choices[0].logprobs.content):
        if (i == 0) or (skip_next):
            if skip_next:
                skip_next = False
            continue
        if '\n' in token_data.token:
            new_logprob.token += token_data.token
            new_logprob.bytes += token_data.bytes
            new_logprob.logprob = min(new_logprob.logprob, token_data.logprob)
            new_logprob.top_logprobs += token_data.top_logprobs
            new_logprobs_list.append(new_logprob)
            new_logprob = temp_response.choices[0].logprobs.content[i + 1]
            skip_next = True
            continue
        if token_data.bytes[0] == 32:
            new_logprobs_list.append(new_logprob)
            new_logprob = token_data
        else:
            new_logprob.token += token_data.token
            new_logprob.bytes += token_data.bytes
            new_logprob.logprob = min(new_logprob.logprob, token_data.logprob)
            new_logprob.top_logprobs += token_data.top_logprobs
        if i == (len(temp_response.choices[0].logprobs.content) - 1):
            new_logprobs_list.append(new_logprob)
    temp_response.choices[0].logprobs.content = new_logprobs_list
    return temp_response


def split(list, n):
    """
    Given a list, it returns a new list of n-sized lists. The items in each
    n-sized list is determined by the order of the original list.

    Args:
        list: a Python list
        n: int

    Return:
        A list of n-sized lists
    """
    return [list[i:i+n] for i in range(0, len(list), n)]


def token_data_group_sequeeze(token_data_list, aggregate_func):
    """
    Given a list of Open AI token data, it squeezes it into one token. All the token will
    be concatenated, the bytes will be concatenated, the logprob will be determined by
    the aggregate_func, and the logprobs will be concatenated.

    Args:
        token_data_list: a list of Open AI token data objects
        aggregate_func: a function that accepts n-numbers of int as it's argument and returns an int

    Returns:
        An Open AI token data object
    """
    new_logprob = copy.deepcopy(token_data_list[0])
    new_logprob.token = ''.join(
        [token_data.token for token_data in token_data_list])
    new_logprob.bytes = [
        byte for token_data in token_data_list for byte in token_data.bytes]
    new_logprob.logprob = aggregate_func(
        [token_data.logprob for token_data in token_data_list])
    new_logprob.top_logprobs = [
        top_logprob for token_data in token_data_list for top_logprob in token_data.top_logprobs]
    if hasattr(token_data_list[0], 'uncertain'):
        new_logprob.uncertain = token_data_list[0].uncertain
    return new_logprob


def sequential_combine(response, mode, aggregate_func=min):
    """
    Given an OpenAI response object with logprobs, combines the token_data list
    into groups of size mode. The method to combine the logprobs value is determined
    by the aggregate_func.

    Args:
        response: OpenAI response object
        mode: int
        aggregate_func: a function that accepts n-numbers of int as it's argument and returns an int

    Returns:
        An OpenAI Response object
    """
    temp_response = copy.deepcopy(response)
    skip_next = False
    logprobs = temp_response.choices[0].logprobs.content

    # split by sentences
    sentences = []
    start = 0
    total_token = 0
    for i, token_data in enumerate(logprobs):
        if ('\n' in token_data.token) or ('.' in token_data.token) or ('?' in token_data.token) or ('!' in token_data.token):
            sentences += [logprobs[start: i + 1]]
            total_token += len(logprobs[start: i + 1])
            start = i + 1
    if total_token != len(logprobs):
        sentences += [logprobs[start:]]
        total_token += len(logprobs[start:])

    log_probs_list = []
    # splits the sentences by words, and group them based on mode
    if isinstance(mode, int):
        for sentence in sentences:
            grouped_sentence = split(sentence, mode)
            for group in grouped_sentence:
                log_probs_list += [
                    token_data_group_sequeeze(group, aggregate_func)]
        temp_response.choices[0].logprobs.content = log_probs_list
    return temp_response

## 1 and 2. Querying the LLM and Combining Tokens
The tokens where the LLM will re-check will first be combined into either a word, a phrase, or a sentence.

### Example 1: Richest Man in Babylon

In [ ]:
response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful and book expert that answers questions about the book "},
        {"role": "user", "content": 'Who is the author of the book "The Richest Man in Babylon"?'},
        {"role": "assistant", "content": 'The author of "The Richest Man in Babylon" is George S. Clason. The book, first published in 1926, offers financial advice through a collection of parables set in ancient Babylon.'},
        {"role": "user", "content": question}
      ],
      logprobs=True,
    )

In [ ]:
print(response.choices[0].message.content)

In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success.


In [ ]:
logprobs_pretty_print(response, question)

Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. 

Tokens:                 In              "            Th

It looks like the LLM is unsure with some of the tokens. However, some of the tokens are not words, for instance, it seems to split "Arkad" into 2 tokens: "Ark" and "ad". Let's combine the tokens that are half words, and set the logprobs as the lowest among them.

In [ ]:
logprobs_pretty_print(combine_token_to_word(response), question)

Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. 

Tokens:                  In            "The         Ric

Now we combine the tokens into groups of N, where the logprobs are determined through an aggregate function like a minimum or a mean.

In [ ]:
print('aggregate by number of words: 7 \naggregate function: min')
logprobs_pretty_print(sequential_combine(combine_token_to_word(response), 7, min), question)

aggregate by number of words: 7 
aggregate function: min
Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. 

In [ ]:
print('aggregate by number of words: 7 \naggregate function: mean')
logprobs_pretty_print(sequential_combine(combine_token_to_word(response), 7, np.mean), question)

aggregate by number of words: 7 
aggregate function: mean
Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success.

Great! We can see that it's actually unsure with a lot of things, maybe because the llm lacks information about these things. Therefore, we can use RAG to specifically query for this.

### Example 2: Joe Biden

In [ ]:
prompt_joe_biden = 'When was joe biden born?'
response_joe_biden = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful assistant who's an expert in famous american figures. If you dont know the person, say 'IDK'"},
        {"role": "user", "content": prompt_joe_biden},
      ],
      logprobs=True,
    )

In [ ]:
logprobs_pretty_print(response_joe_biden, prompt_joe_biden)

Prompt:     When was joe biden born?
Response:   Joe Biden was born on November 20, 1942. 

Tokens:           Joe     Biden       was      born        on  November                  20         ,                 194         2         .
Logprobs:        0.00      0.00      0.00      0.00      0.00     -0.00      0.00      0.00      0.00      0.00      0.00      0.00     -0.00
Linprob:      100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%
Perplexity: 1.0000001764987771 



In [ ]:
logprobs_pretty_print(combine_token_to_word(response_joe_biden), prompt_joe_biden)

Prompt:     When was joe biden born?
Response:   Joe Biden was born on November 20, 1942. 

Tokens:           Joe     Biden       was      born        on  November       20,     1942.
Logprobs:        0.00      0.00      0.00      0.00      0.00     -0.00      0.00     -0.00
Linprob:      100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%   100.00%
Perplexity: 1.0000002868105287 



### Example 3: Unknown Person

In [ ]:
prompt_unknown_person = 'When was jake amber born?'
response_unknown_person = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful assistant who's an expert in famous american figures. If you don't know the person, make up random innformations."},
        {"role": "user", "content": prompt_unknown_person},
      ],
      logprobs=True,
    )

In [ ]:
logprobs_pretty_print(response_unknown_person, prompt_unknown_person)

Prompt:     When was jake amber born?
Response:   I'm sorry, but I don't have any information on a figure named Jake Amber. It's possible he could be a fictional character or a less-known individual. If you have any other questions about more prominent American figures or historical personalities, feel free to ask! 

Tokens:                I'm          sorry              ,            but              I          don't           have            any    information             on              a         figure          named           Jake          Amber              .           It's       possible             he          could             be              a      fictional      character             or              a           less         -known     individual              .             If            you           have            any          other      questions          about           more      prominent       American        figures             or     historical  personalities          

In [ ]:
logprobs_pretty_print(combine_token_to_word(response_unknown_person), prompt_unknown_person)

Prompt:     When was jake amber born?
Response:   I'm sorry, but I don't have any information on a figure named Jake Amber. It's possible he could be a fictional character or a less-known individual. If you have any other questions about more prominent American figures or historical personalities, feel free to ask! 

Tokens:                 I'm          sorry,             but               I           don't            have             any     information              on               a          figure           named            Jake          Amber.            It's        possible              he           could              be               a       fictional       character              or               a      less-known     individual.              If             you            have             any           other       questions           about            more       prominent        American         figures              or      historical  personalities,            feel            fr

In [ ]:
logprobs_pretty_print(sequential_combine(combine_token_to_word(response_unknown_person), 4, np.mean), prompt_unknown_person)

Prompt:     When was jake amber born?
Response:   I'm sorry, but I don't have any information on a figure named Jake Amber. It's possible he could be a fictional character or a less-known individual. If you have any other questions about more prominent American figures or historical personalities, feel free to ask! 

Tokens:                         I'm sorry, but I           don't have any information                    on a figure named                          Jake Amber.               It's possible he could             be a fictional character          or a less-known individual.                      If you have any           other questions about more        prominent American figures or  historical personalities, feel free                              to ask!
Logprobs:                                  -0.65                                -0.42                                -0.27                                -0.00                                -1.08                             

We can see that it's very unsure with a lot of things when it makes thigns up



## 2. Annotating


In [ ]:
def annotater(response, tolerance=-0.4):
    """
    Given an OpenAI response object with logprobs, marks all the tokens
    where the logprob is below the tolerance as uncertain by adding a field
    called uncertain and marking it as True.

    Args:
        response: OpenAI response object
        tolerance: int

    Returns:
        An OpenAI response object
    """
    temp_response = copy.deepcopy(response)
    for token_data in temp_response.choices[0].logprobs.content:
        if token_data.logprob < tolerance:
            token_data.uncertain = True
        else:
            token_data.uncertain = False
    return temp_response


def annotated_combiner(response, aggregate_func=np.mean):
    """
    Given a OpenAI response object with logprobs, combines all adjacent
    token data objects in the list of response into one token data object,
    aggregated with aggregate_func.

    Args:
        response: OpenAI Response object
        aggregate_func: a function that accepts n-numbers of int as it's argument and returns an int

    Returns:
        An OpenAI response object
    """
    temp_response = copy.deepcopy(response)
    index_groups = []
    current_group = []
    for token_data in temp_response.choices[0].logprobs.content:
        if token_data.uncertain:
            if ('\n' not in token_data.token) and ('.' not in token_data.token) and ('?' not in token_data.token) and ('!' not in token_data.token):
                current_group += [token_data]
            else:
                if len(current_group) > 0:
                    index_groups += [current_group]
                    current_group = []
                index_groups += [[token_data]]
        else:
            if len(current_group) > 0:
                index_groups += [current_group]
                current_group = []
                index_groups += [[token_data]]
                continue
            index_groups += [[token_data]]
    log_probs_list = []
    for group in index_groups:
        log_probs_list += [token_data_group_sequeeze(group, aggregate_func)]
    temp_response.choices[0].logprobs.content = log_probs_list
    return temp_response


def uncertain_marker(response):
    """
    Given an OpenAI response object with logprobs, modifies all the logprobs token data list
    strings where uncertain is set to True with '[uncertain]' + token + '[/uncertain]'.

    Args:
        response: OpenAI Object

    Returns:
        An OpenAI response object
    """
    temp_response = copy.deepcopy(response)
    if not hasattr(response.choices[0].logprobs.content[0], 'uncertain'):
        temp_response = annotated_combiner(annotater(temp_response))
    for token_data in temp_response.choices[0].logprobs.content:
        if token_data.uncertain:
            token_data.token = ' ([uncertain]' + \
                token_data.token + ' [/uncertain]) '
    temp_response.choices[0].message.content = ''.join(
        [i.token for i in temp_response.choices[0].logprobs.content])
    return temp_response

In [ ]:
logprobs_pretty_print(sequential_combine(combine_token_to_word(response), 5, np.mean), question)

Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. 

Tokens:                      In "The Richest Man in    

Now let's for all the tokens(sequence of tokens) where the logprobs is below -0.4, we mark them as uncertain and add them as a field

In [ ]:
annotated_rmib = annotater(sequential_combine(combine_token_to_word(response), 5, np.mean), tolerance= -0.4)
logprobs_pretty_print(annotated_rmib, question)

Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. 

Tokens:                      In "The Richest Man in    

It seems that there a lot of adjacent sequences of tokens where the LLM is uncertain, it would make sense to combine them into one sequence of tokens and aggregate the logprob with an aggregate function.

In [ ]:
annotated_combined = annotated_combiner(annotated_rmib, np.mean)
logprobs_pretty_print(annotated_combined, question)

Prompt:     Why did Arkad believe that good luck follows opportunity?
Response:   In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success. 

Tokens:                                                

Now, let's combine all the tokens into a single string and mark the uncertain ones with [uncertain] [/uncertain]

In [ ]:
print(response.choices[0].message.content)

In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he sees luck as a byproduct of one’s readiness to seize opportunities when they arise. He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the opportunities presented to them. 

Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter opportunities that lead to success. In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take risks, and the ability to recognize and act on chances that life presents. This perspective encourages readers to be proactive and to seek out opportunities, rather than relying solely on chance for financial success.


In [ ]:
marked_response = uncertain_marker(annotated_combined)
print(marked_response.choices[0].message.content)

In "The Richest Man in ([uncertain] Babylon," Arkad, who is the [/uncertain])  richest man in Babylon, believes that good luck follows opportunity because he sees luck as ([uncertain] a byproduct of one’s readiness [/uncertain])  to seize opportunities when they arise. ([uncertain] He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the [/uncertain])  opportunities presented to them. 

 ([uncertain]Arkad suggests that those who are diligent, prepared, and willing to work toward their goals [/uncertain])  are more likely to encounter ([uncertain] opportunities that lead to success. [/uncertain])  ([uncertain] In essence, he emphasizes that luck is not merely random chance; rather, it is created through effort, willingness to take [/uncertain])  risks, and the ability to ([uncertain] recognize and act on chances [/uncertain])  ([uncertain] that life presents. [/uncertain])  ([uncertain] This perspective encourages readers to be pro

## 3. Constructing questions to get a more confident answer for that token.

first we need to replace the uncertain tokens with the question.

In [ ]:
marked_response_question_replaced = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": """You are a book expert that answers questions about books. """},
        {"role": "user", "content": question},
        {"role": "assistant", "content": response.choices[0].message.content},
        {"role": "system", "content": f"""The answer has been marked to where you are uncertain with the phrases. For every, phrases in between
        [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)]

        First example:
        user: What is meaning of the colors in the flag of Ghana?
        assistant: Red is for the blood of martyrs, green for forests, and gold for mineral wealth.
        user: Here's the annotated version: ([uncertain] Red [/uncertain]) is for the blood of martyrs, ([uncertain] green for forests [/uncertain]), and gold for mineral wealth.
        assistant: [Search(is red a color in the flag of Ghana?)] is for the blood of martyrs, [Search(is green a color in the flag of Ghana? If so, what does it symbolize?)], and gold for mineral wealth.

        Second example:
        user: Give me a very short summary of Joe Biden's journey becoming the president!
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on August 18, 2019. His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        user: Here's an annotated version: Joe Biden announced his candidacy for the 2020 presidential election on ([uncertain] August 18, 2019 [/uncertain]). His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on [Search(When did Joe Biden announce his candidancy for the 2020 presidential election?)].  His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        """},
        {"role": "user", "content": f"Here's the annotated version: {marked_response.choices[0].message.content}"},
      ],
      logprobs=True,
    )

In [ ]:
print(marked_response_question_replaced.choices[0].message.content)

In "The Richest Man in [Search(What is the setting of the book "The Richest Man in Babylon"?)], Arkad, who is the [Search(Who is Arkad in the context of the book?)] richest man in Babylon, believes that good luck follows opportunity because he sees luck as [Search(What does Arkad mean by "a byproduct of one’s readiness" in terms of seizing opportunities?)] to seize opportunities when they arise. [Search(Why do people miss chances to become wealthy according to Arkad?)] He explains that many people often miss chances to become wealthy because they fail to recognize or act upon the [Search(What type of opportunities does Arkad refer to in the book?)] opportunities presented to them.

[Search(What characteristics do diligent, prepared, and hardworking people exhibit according to Arkad?)] Arkad suggests that those who are diligent, prepared, and willing to work toward their goals are more likely to encounter [Search(What are examples of opportunities that lead to success in the book?)] opp

Now we extract those questions into a dictionary, so we can answer each of them individually with RAG

In [ ]:
question_extracted = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": """You are a book expert that answers questions about books. """},
        {"role": "user", "content": question},
        {"role": "assistant", "content": response.choices[0].message.content},
        {"role": "system", "content": f"""The answer has been marked to where you are uncertain with the phrases. For every, phrases in between
        [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)]

        First example:
        user: What is meaning of the colors in the flag of Ghana?
        assistant: Red is for the blood of martyrs, green for forests, and gold for mineral wealth.
        user: Here's the annotated version: ([uncertain] Red [/uncertain]) is for the blood of martyrs, ([uncertain] green for forests [/uncertain]), and gold for mineral wealth.
        assistant: [Search(is red a color in the flag of Ghana?)] is for the blood of martyrs, [Search(is green a color in the flag of Ghana? If so, what does it symbolize?)], and gold for mineral wealth.

        Second example:
        user: Give me a very short summary of Joe Biden's journey becoming the president!
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on August 18, 2019. His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        user: Here's an annotated version: Joe Biden announced his candidacy for the 2020 presidential election on ([uncertain] August 18, 2019 [/uncertain]). His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on [Search(When did Joe Biden announce his candidancy for the 2020 presidential election?)].  His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        """},
        {"role": "user", "content": f"Here's the annotated version: {marked_response.choices[0].message.content}"},
        {"role": "assistant", "content": marked_response_question_replaced.choices[0].message.content},
        {'role': "user", "content": """Now for all the questions marked as [Search(question)], please extract them in a python dictionary format:
        {
          "1": "question 1",
          "2": "question 2",
          ...
          "n": "question n"
        }
        it is critical to only output the dictionary and nothing else.
        """}
      ],
    )

In [ ]:
print(question_extracted.choices[0].message.content)

{
  "1": "What is the setting of the book 'The Richest Man in Babylon'?",
  "2": "Who is Arkad in the context of the book?",
  "3": "What does Arkad mean by 'a byproduct of one’s readiness' in terms of seizing opportunities?",
  "4": "Why do people miss chances to become wealthy according to Arkad?",
  "5": "What type of opportunities does Arkad refer to in the book?",
  "6": "What characteristics do diligent, prepared, and hardworking people exhibit according to Arkad?",
  "7": "What are examples of opportunities that lead to success in the book?",
  "8": "What does Arkad say about the nature of luck?",
  "9": "What type of risks does Arkad encourage people to take?",
  "10": "How does one recognize and act on chances?",
  "11": "What does Arkad mean by 'chances' in the context of life?",
  "12": "What does Arkad mean by 'that life presents'?",
  "13": "Why does Arkad advocate for a proactive approach to achieving financial success?",
  "14": "What does financial success mean in the c

In [ ]:
questions_dict = ast.literal_eval(question_extracted.choices[0].message.content)
questions_dict

{'1': "What is the setting of the book 'The Richest Man in Babylon'?",
 '2': 'Who is Arkad in the context of the book?',
 '3': "What does Arkad mean by 'a byproduct of one’s readiness' in terms of seizing opportunities?",
 '4': 'Why do people miss chances to become wealthy according to Arkad?',
 '5': 'What type of opportunities does Arkad refer to in the book?',
 '6': 'What characteristics do diligent, prepared, and hardworking people exhibit according to Arkad?',
 '7': 'What are examples of opportunities that lead to success in the book?',
 '8': 'What does Arkad say about the nature of luck?',
 '9': 'What type of risks does Arkad encourage people to take?',
 '10': 'How does one recognize and act on chances?',
 '11': "What does Arkad mean by 'chances' in the context of life?",
 '12': "What does Arkad mean by 'that life presents'?",
 '13': 'Why does Arkad advocate for a proactive approach to achieving financial success?',
 '14': 'What does financial success mean in the context of the bo

## 4. Answering These Questions

In [ ]:
# creating a retriever to answer these questions
recursive_text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n", " "],
    chunk_size = 1200,
    chunk_overlap = 100,
    # length_function = len,
    is_separator_regex=False
)

splits = []
for i in range(len(df_metadata)):
  if df_metadata.loc[df_metadata.index[i]].type == 'full text':
    continue
  temp_splits = recursive_text_splitter.create_documents(texts=[df_metadata.loc[df_metadata.index[i]].text])
  for temp_split in temp_splits:
    temp_split.page_content = f'chapter: {df_metadata.loc[df_metadata.index[i]].title}\ntype: {df_metadata.loc[df_metadata.index[i]].type}\n\n' + temp_split.page_content
    splits.append(temp_split)


embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    collection_name="rmib",
    embedding_function=embeddings,
)
vector_store.add_documents(documents=splits, ids=[f'id_{i}' for i in range(1, len(splits) + 1)]);
retriever = vector_store.as_retriever(search_kwargs={"k": 8})

In [ ]:
message = [
        {"role": "system", "content": f"""You are a book expert that answers questions about books.
      Question: {question}
      Context: called the RAG
      Original Answer: {response.choices[0].message.content}

      Now, I have marked the answer to where you are uncertain with the phrases. For every, phrases in between
      [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)]

      Annotated Answer: {marked_response.choices[0].message.content}
      Constructed Questions Answer: {marked_response_question_replaced.choices[0].message.content}
      """},
    ]

In [ ]:
constructed_question_answer = {}
for i in range(1, len(questions_dict) + 1):
    question_temp = questions_dict[str(i)]

    # getting the context for the question
    context = retriever.get_relevant_documents(question_temp)

    # constructing the question and context message
    question_string = f"""Use the following pieces of retrieved context to answer the question.
    Question: {question}
    Context: {format_docs(context)}
    Answer:
    """
    question_message = message + [{"role": "user", "content": question_string}]

    # answering the question
    question_answer = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=question_message,
    )
    constructed_question_answer[str(i)] = [
        question_temp, question_answer.choices[0].message.content]

<ipython-input-45-e92dba8e0836>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  context = retriever.get_relevant_documents(question_temp)


## 5. Reconstructing the Answer

In [ ]:
question_answer = ""
for i in range(1, len(questions_dict) + 1):
    question_answer += f"""Question: {questions_dict[str(i)]}\nAnswer: {constructed_question_answer[str(i)][1]}\n"""
print(question_answer)

Question: What is the setting of the book 'The Richest Man in Babylon'?
Answer: Arkad believed that good luck follows opportunity because he understood that wealth is not simply a matter of fate or luck; it is a result of taking action when opportunities present themselves. During his journey to accumulate wealth, he realized that many people fail to recognize the opportunities available to them or choose not to act on them. Instead of waiting for fortune to smile upon them, Arkad emphasized the importance of being proactive, diligent, and prepared to seize chances when they arise. 

According to Arkad, those who actively seek opportunities, educate themselves about how to manage and grow their wealth, and are willing to work and invest wisely are more likely to create their own "luck." This philosophy is encapsulated in his advice that a part of all one earns should be kept for oneself, which reflects the notion that individuals have the power to influence their financial destiny by b

In [ ]:
# reconstruction message
reconstructing = f"""
    Here are the questions and their answers:
    {question_answer}
    Now with answers to those questions and the original question: {question}, improve the original answer without changing the format of the answer.

    Notes:
    It's critical to just output the final answer.
    It's critical to not output the annotated answer.
    It's critical to not output constructed questions answer.

    Final Answer:
    """
reconstructing_message = message + [{"role": "user", "content": reconstructing}]

In [ ]:
reconstructed_answer = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=reconstructing_message,
)

In [ ]:
print(reconstructed_answer.choices[0].message.content)

In "The Richest Man in Babylon," Arkad, who is the richest man in Babylon, believes that good luck follows opportunity because he understands that wealth is not simply a matter of fate or luck; it is a result of taking action when opportunities present themselves. Throughout his life, he observed that many people fail to recognize the opportunities available to them or choose not to act on them. Instead of waiting for fortune to smile upon them, Arkad emphasized the importance of being proactive, diligent, and prepared to seize chances when they arise.

According to Arkad, those who actively seek opportunities, educate themselves about how to manage and grow their wealth, and are willing to work and invest wisely are more likely to create their own "luck." He noted that while opportunities are available to everyone, only a few grasp them and achieve their desires, while the majority hesitate or falter and consequently fall behind. This perspective underscores that luck is not merely ra

# FLARE Function

In [ ]:
def flare(question, retriever, openai_api_key, openai_model='gpt-4o-mini', tolerance=-0.4, verbose=True):
    '''
    Uses a advanced RAG technique called FLARE to answer the question. It's an implementation
    of the paper: "Active Retrieval Augmented Generation" Jiang ZB and fellow scientists in October
    2023.

    Args:
      question: The question to be answered.
      retriever: langchain retriever to be used.
      openai_api_key: The OPENAI API key to be used.
      openai_model: OpenAI Model to use
      tolerance: The tolerance of logprobs to be marked as uncertain
    Returns:
      A 3-item tuple, which are all OpenAI objects. The items are the original answer,
      annotated answer, and finally the improved answer.

    '''
    client = OpenAI(api_key=openai_api_key)

    # getting the first output, normal rag
    # get context
    context = retriever.get_relevant_documents(question)

    # constructing message
    message = [
        {"role": "system", "content": """You are a book expert that answers questions about books. Use the following pieces of retrieved context to answer the question."""},
        {"role": "user", "content": f"""
        Context: {format_docs(context)}
        Question: {question}
        Answer:
        """}
    ]

    if verbose: print(f'Acquiring answer with traditional RAG...')
    # answer the question
    answer = client.chat.completions.create(
        model=openai_model,
        messages=message,
        logprobs=True,
    )

    if verbose: print(f'Finding uncertain tokens, and annotating the answer...')
    # annotate the question
    annotated_answer = uncertain_marker(annotated_combiner(annotater(sequential_combine(
        combine_token_to_word(answer), 5, np.mean), tolerance=tolerance), np.mean))

    # constructing the questions for the uncertained answers
    message += [
        {"role": "assistant", "content": answer.choices[0].message.content},
        {"role": "system", "content": f"""Now, I have marked the answer to where you are uncertain with the phrases. For every, phrases in between
        [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)].

        This question is going to be used independently to get get relevant texts from a vector database
        It's critical that the question includes the object and subject of the phrase
        It's critical that the question has context about the annswer


        First example:
        user: What is meaning of the colors in the flag of Ghana?
        assistant: Red is for the blood of martyrs, green for forests, and gold for mineral wealth.
        user: Here's the annotated version: ([uncertain] Red [/uncertain]) is for the blood of martyrs, ([uncertain] green for forests [/uncertain]), and gold for mineral wealth.
        assistant: [Search(is red a color in the flag of Ghana?)] is for the blood of martyrs, [Search(is green a color in the flag of Ghana? If so, what does it symbolize?)], and gold for mineral wealth.

        Second example:
        user: Give me a very short summary of Joe Biden's journey becoming the president!
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on August 18, 2019. His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        user: Here's an annotated version: Joe Biden announced his candidacy for the 2020 presidential election on ([uncertain] August 18, 2019 [/uncertain]). His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        assistant: Joe Biden announced his candidacy for the 2020 presidential election on [Search(When did Joe Biden announce his candidancy for the 2020 presidential election?)].  His campaign focused on issues such as restoring the 'soul of America', expanding healthcare access, and addressing climate change.
        """},
        {"role": "user",
            "content": f"Here's the annotated version: {annotated_answer.choices[0].message.content}"},
    ]

    if verbose: print(f'Constructing questions for the annotated tokens...')
    questions_construction = client.chat.completions.create(
        model=openai_model,
        messages=message,
    )

    # extracting the questions
    message += [
        {"role": "assistant",
            "content": questions_construction.choices[0].message.content},
        {'role': "user", "content": """Now for all the questions marked as [Search(question)], please extract them in a python dictionary format:
        {
        "1": "question 1",
        "2": "question 2",
        ...
        "n": "question n"
        }

        It is critical to only output the dictionary and nothing else.
        It is critical to not output it in a markdown format.
        It is critical that the first character of the output starts with an open curly bracket '{'
      """}
    ]

    if verbose: print(f'Extracting constructed questions...')
    questions = client.chat.completions.create(
        model=openai_model,
        messages=message,
    )
    retry_count= 1
    try:
        questions_dict = ast.literal_eval(questions.choices[0].message.content)
    except:
        while retry_count <= 3:
            if verbose: print(f"Couldn't convert to dictionary, attempting to fix the dictionary. Retry count: {retry_count}")
            questions = client.chat.completions.create(
                model=openai_model,
                messages=message,
            )
            try:
                questions_dict = ast.literal_eval(questions.choices[0].message.content)
                break
            except:
                retry_count += 1
                continue
        else:
            print(f'FLARE Failed, try to call the function again.')
            return

    # message to answer the questions one by one
    new_message = [
        {"role": "system", "content": f"""You are a book expert that answers questions about books.
      Question: {question}
      Context: called the RAG
      Original Answer: {answer.choices[0].message.content}

      Now, I have marked the answer to where you are uncertain with the phrases. For every, phrases in between
      [uncertain] [/uncertain], please construct a question that will answer each uncertain phrase and mark it as [Search(question)]

      Annotated Answer: {annotated_answer.choices[0].message.content}
      Constructed Questions Answer: {questions_construction.choices[0].message.content}
      """},
    ]

    if verbose: print(f'Answering each questions...')
    # answering each of the questions one by one
    constructed_question_answer = {}
    for i in range(1, len(questions_dict) + 1):
        question_temp = questions_dict[str(i)]

        # getting the context for the question
        context = retriever.get_relevant_documents(question_temp)

        # constructing the question and context message
        question_string = f"""Use the following pieces of retrieved context to answer the question.
        Question: {question}
        Context: {format_docs(context)}
        Answer:
        """
        question_message = new_message + \
            [{"role": "user", "content": question_string}]

        # answering the question
        question_answer = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=question_message,
        )
        constructed_question_answer[str(i)] = [
            question_temp, question_answer.choices[0].message.content]

    # reconstructing to get the final answer
    question_answer = ""
    for i in range(1, len(questions_dict) + 1):
        question_answer += f"""Question: {questions_dict[str(i)]}\nAnswer: {constructed_question_answer[str(i)][1]}\n"""

    reconstructing = f"""
    Here are the questions and their answers:
    {question_answer}
    Now with answers to those questions and the original question: {question}, improve the original answer without changing the format of the answer.

    Notes:
    It's critical to just output the final answer.
    It's critical to not output the annotated answer.
    It's critical to not output constructed questions answer.

    Final Answer:
    """
    reconstructing_message = new_message + \
        [{"role": "user", "content": reconstructing}]
    if verbose: print(f'Reconstructing the final answer...')
    reconstructed_answer = client.chat.completions.create(
        model=openai_model,
        messages=reconstructing_message,
    )

    # printing some information
    # It prints the question construction for the annotated answer because it's clearer to make the annotations as the questions itself
    return answer, questions_construction, reconstructed_answer

In [ ]:
print(df_questions.loc['the sixth cure', 1].hard)

How can a man ensure a future income according to the passage?


In [ ]:
original_answer_1, annotated_answer_1, final_answer_1 = flare(df_questions.loc['the sixth cure', 1].hard, retriever, userdata.get('OPENAI_API_KEY'), verbose=False)

In [ ]:
print(final_answer_1.choices[0].message.content)

According to the passage from "The Sixth Cure," a man can ensure a future income by making careful preparations for his financial future, especially as he grows older. The key steps he can take include:

1. **Preparation for Future Needs**: A man should make thoughtful preparations for a suitable income during his later years and for his family in case he is no longer able to support them.

2. **Creating a Growing Surplus**: He should focus on acquiring a surplus of wealth that will allow him to invest in provisions or assets that will endure over time.

3. **Investing Wisely**: The man is advised to carefully invest his treasure, avoiding risky ventures that promise high returns but may lead to significant losses. Instead, he should seek stable and secure investments.

4. **Regular Contributions**: It’s recommended to make small, regular payments into protective plans or investments that can provide for future needs, ensuring family security in his absence.

5. **Consulting Knowledgea

All functions and datasets used have been placed in https://github.com/Justin-Jonany/FLARE_Implementation/tree/main